In [1]:
import torch
from torch import nn
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import numpy as np
import cv2
from glob import glob
import os
import tqdm
from GoogLeNet import GoogLeNet, InceptionBlock, ConvBlock, evaluate, accuracy

device='cpu'

In [2]:
x = torch.randn(1, 3, 224, 224)
model = GoogLeNet()

out = model(x)
out.shape

torch.Size([1, 10])

In [3]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='../datasets/', train=True, transform=transforms, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../datasets/', train=False, transform=transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [6]:
def fit(epochs, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), 6e-5)
    for epoch in range(epochs):
        lrs = []
        loss = 0
        acc = 0
        for batch in tqdm.tqdm(train_loader):
            loss, acc = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, loss, acc))
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [7]:
device = 'cuda'

In [8]:
model = model.to(device)

In [9]:
import tqdm
history = fit(5, model, train_dl, test_dl)

100%|██████████| 782/782 [04:23<00:00,  2.97it/s]


Epoch [0], loss: 1.1989, acc: 0.5625
Epoch [0], val_loss: 1.0540, val_acc: 0.6253


100%|██████████| 782/782 [04:23<00:00,  2.96it/s]


Epoch [1], loss: 0.6828, acc: 0.8125
Epoch [1], val_loss: 0.8216, val_acc: 0.7095


100%|██████████| 782/782 [04:21<00:00,  2.99it/s]


Epoch [2], loss: 0.7122, acc: 0.6875
Epoch [2], val_loss: 0.6891, val_acc: 0.7581


100%|██████████| 782/782 [04:23<00:00,  2.96it/s]


Epoch [3], loss: 0.7635, acc: 0.7500
Epoch [3], val_loss: 0.6549, val_acc: 0.7796


100%|██████████| 782/782 [04:20<00:00,  3.00it/s]


Epoch [4], loss: 1.1769, acc: 0.5625
Epoch [4], val_loss: 0.6196, val_acc: 0.7966
